In [1]:
#importing libraries
import pandas as pd
import nltk 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import re 

In [2]:
#Reading the trainig data
data = pd.read_csv('C:\\Users\\warnak\\OneDrive - John Keells Holdings PLC\\workspace\\msc\\ML\\train.txt', sep='\t')

In [3]:
#reading the test data
test = pd.read_csv('C:\\Users\\warnak\\OneDrive - John Keells Holdings PLC\\workspace\\msc\\ML\\test.csv')

# Pre-Processing steps

In [4]:
#combining the 2 datasets for preprocessing 
combi = data.append(test, ignore_index=True)

C:\Anaconda3\envs\msc_py35\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [5]:
#function to remove a given pattern in a string
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt    

In [6]:
#Removing mentions in tweets
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['Tweet text'], "@[\w]*")

In [7]:
#Removing special characters, numbers, punctuations
combi['tidy_tweet'] = combi['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [8]:
#Removing the which have less than 3 characters 
combi['tidy_tweet'] = combi['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [9]:
#tokenizing the tweets 
tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [Sweet, United, Nations, video, Just, time, Ch...
1    [rumored, have, talked, agent, Angels, asked, ...
2      [there, Nice, Minnesota, Winter, Weather, #Not]
3                  [episodes, left, dying, over, here]
4    [breathe, chosen, most, notable, quote, year, ...
Name: tidy_tweet, dtype: object

In [10]:
#Performing stemming 
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [sweet, unit, nation, video, just, time, chris...
1    [rumor, have, talk, agent, angel, ask, about, ...
2      [there, nice, minnesota, winter, weather, #not]
3                      [episod, left, die, over, here]
4    [breath, chosen, most, notabl, quot, year, ann...
Name: tidy_tweet, dtype: object

In [11]:
#adding the tokenized tweets 
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['tidy_tweet'] = tokenized_tweet

In [12]:
#creating a tfidf vector
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(combi['tidy_tweet'])

# Model Building

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_tfidf = tfidf[:3817,:]
test_tfidf = tfidf[3817:,:]

# splitting data into training and validation set
xtrain_tfidf, xvalid_tfidf, ytrain, yvalid = train_test_split(train_tfidf, data['Label'], random_state=42, test_size=0.3)

lreg = LogisticRegression()
lreg.fit(xtrain_tfidf, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_tfidf) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

0.7446524064171124

# Getting the predictions

In [14]:
test_pred = lreg.predict_proba(test_tfidf)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['Label'] = test_pred_int

In [15]:
submission = test[['Tweet index','Label']]
submission.to_csv('sub_lreg_tfidf.csv', index=False) # writing data to a CSV file